In [1]:
from keras.datasets import cifar10
from keras.models import load_model
from keras.utils.np_utils import to_categorical
import numpy as np
from six.moves import cPickle as pickle
import os.path
from IPython import display
import matplotlib.pyplot as plt

% matplotlib inline
% config InlineBackend.figure_format = 'retina'

Using TensorFlow backend.


In [2]:
def prepData(image_class):
    # Fetch data
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()

    # Convert uint8 pixel values to float32 in the range [0, 1] (for sigmoid)
    X_test = X_test.astype('float32')
    X_test /= 255

    # Split dataset
    X_new = np.ndarray(shape=(X_test.shape[0] / 10, X_test.shape[1], 
                        X_test.shape[2], X_test.shape[3]), dtype=np.float32)
    X_other = np.ndarray(shape=(9 * X_test.shape[0] / 10, X_test.shape[1], 
                        X_test.shape[2], X_test.shape[3]), dtype=np.float32)

    sampleNumber = 0
    otherNumber = 0
    for X, y in zip(X_test, y_test):
        if y[0] == image_class:
            X_new[sampleNumber] = X
            sampleNumber += 1
        else:
            X_other[otherNumber] = X
            otherNumber += 1
    X_test = X_new[:, :, :, :]

    return X_test, X_other

In [4]:
for epoch in [50, 100, 150, 200, 250, 300]:
    avg_accuracy = 0
    for image_class in range(10):
        X_test, X_other = prepData(image_class)
        y_test = np.zeros(shape=(X_test.shape[0], 2))
        y_other = np.zeros(shape=(X_other.shape[0], 2))
        y_test[:, 0] = 1 # Real class images
        y_other[:, 1] = 1 # Real other images

        other_indices = np.random.randint(0, X_other.shape[0], X_test.shape[0])
        X_test = np.concatenate((X_test, X_other[other_indices, :, :, :]))
        y_test = np.concatenate((y_test, y_other[other_indices, :]))

        discriminator = load_model("networks/disc" + str(image_class) + "-" + str(epoch) + ".h5")

        preds = discriminator.predict(X_test)

        accuracy = 0
        for pred_index in range(len(preds)):
            if y_test[pred_index, 0] == 1:
                if preds[pred_index, 0] >= preds[pred_index, 1]:
                    accuracy += 1
            elif y_test[pred_index, 1] == 1:
                if preds[pred_index, 0] <= preds[pred_index, 1]:
                    accuracy += 1
            else:
                print "y_test should not include generated images"

        accuracy /= 1. * len(preds)

        msg = "Epoch: {:d}, Image class: {:d}".format(epoch, image_class)
        msg += ", Accuracy: {:.4f}".format(accuracy)
        print msg

        avg_accuracy += accuracy
    print "\nEpoch: {:d} Average Accuracy: {:.4f}\n".format(epoch, avg_accuracy / 10)

Epoch: 50, Image class: 0, Accuracy: 0.9070
Epoch: 50, Image class: 1, Accuracy: 0.9550
Epoch: 50, Image class: 2, Accuracy: 0.8570
Epoch: 50, Image class: 3, Accuracy: 0.8125
Epoch: 50, Image class: 4, Accuracy: 0.8800
Epoch: 50, Image class: 5, Accuracy: 0.8700
Epoch: 50, Image class: 6, Accuracy: 0.9175
Epoch: 50, Image class: 7, Accuracy: 0.8960
Epoch: 50, Image class: 8, Accuracy: 0.9415
Epoch: 50, Image class: 9, Accuracy: 0.9285

Epoch: 50 Average Accuracy: 0.8965

Epoch: 100, Image class: 0, Accuracy: 0.9065
Epoch: 100, Image class: 1, Accuracy: 0.9550
Epoch: 100, Image class: 2, Accuracy: 0.8485
Epoch: 100, Image class: 3, Accuracy: 0.8110
Epoch: 100, Image class: 4, Accuracy: 0.8915
Epoch: 100, Image class: 5, Accuracy: 0.8490
Epoch: 100, Image class: 6, Accuracy: 0.9055
Epoch: 100, Image class: 7, Accuracy: 0.9080
Epoch: 100, Image class: 8, Accuracy: 0.9370
Epoch: 100, Image class: 9, Accuracy: 0.9275

Epoch: 100 Average Accuracy: 0.8940

Epoch: 150, Image class: 0, Accurac

In [3]:
# Fetch data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Convert uint8 pixel values to float32 in the range [0, 1] (for sigmoid)
X_test = X_test.astype('float32')
X_test /= 255

for epoch in [50, 100, 150, 200, 250, 300]:
    all_class_preds = np.ndarray(shape=(X_test.shape[0], 10), dtype=np.float32)
    for image_class in range(10):
        discriminator = load_model("networks/disc" + str(image_class) + "-" + str(epoch) + ".h5")
        preds = discriminator.predict(X_test)
        class_preds = preds[:, 0]
        all_class_preds[:, image_class] = class_preds

    # Calculate joint accuracy
    all_class_preds_index = np.argmax(all_class_preds, axis=1)
    accuracy = 0
    for y, p in zip(y_test, all_class_preds_index):
        if y == p:
            accuracy += 1
    accuracy /= 1. * len(y_test)
    print "Epoch {:d} joint accuracy: {:.4f}".format(epoch, accuracy)

Epoch 50 joint accuracy: 0.7648
Epoch 100 joint accuracy: 0.7921
Epoch 150 joint accuracy: 0.7943
Epoch 200 joint accuracy: 0.8049
Epoch 250 joint accuracy: 0.8032
Epoch 300 joint accuracy: 0.8032
